In [1]:
import numpy as np
import scipy as sp
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState
from sym import Rot3

In [2]:
def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([
        [ q3,  q2, -q1, -q0],
        [-q2,  q3,  q0, -q1],
        [ q1, -q0,  q3, -q2]
    ])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [3]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation. Refer to this source for derivation:
        # https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)
                                
p = Params()

In [4]:
class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        super().__init__()

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        thrust = p.k_thrust * np.square(rotor_rates)
        return thrust

    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        thrusts = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates)
        
        u1 = np.array([0, 0, np.sum(thrusts)])
        
        u2 = np.array([
            p.arm_length * (thrusts[0] - thrusts[2]),
            p.arm_length * (thrusts[1] - thrusts[3]),
            p.k_torque * (input.rotor_rates[0] - input.rotor_rates[1] + input.rotor_rates[2] - input.rotor_rates[3])
        ])
        
        def state_derivative(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2
            ).to_state_vector()
        
        state_vector = self.state.to_state_vector()
        
        solution = sp.integrate.solve_ivp(
            fun=state_derivative,
            t_span=(0, self.dt),
            y0=state_vector,
            method='RK45'
        )
        
        self.state = QuadrotorState.from_state_vector(solution.y[:, -1])
        return self.state

    @staticmethod
    def state_derivative(state: QuadrotorState, u1: np.ndarray, u2: np.ndarray) -> QuadrotorState:
        R = state.orientation.to_rotation_matrix()
        gravity = np.array([0, 0, -p.g])
        
        accel = (1 / p.mass) * (R @ u1 + p.mass * gravity)
        
        omega = state.angular_velocity
        angular_accel = np.linalg.inv(p.inertia) @ (u2 - np.cross(omega, p.inertia @ omega))
        
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, omega),
            angular_velocity=angular_accel
        )
        
        return state_derivative

In [5]:
required_thrust = p.mass * p.g

# Calculating the rotor rate needed to produce half of the required thrust (as there are 4 rotors)
hovering_rotor_rate = np.sqrt(required_thrust / (4 * p.k_thrust))
print(hovering_rotor_rate);

419.9395843340998


In [6]:
class EmptyTrajectory(TrajectoryBase):
    def eval(self, *args):
        return TrajectoryState(0.0, position=np.zeros(3))  # Trajectory not yet important

In [7]:
class EmptyController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(np.array([300, 300, 300, 300]))

In [8]:
class HoveringController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(np.array([hovering_rotor_rate, hovering_rotor_rate, hovering_rotor_rate, hovering_rotor_rate]))

In [9]:
from quadrotor.simulator import SimulatorBase, SimulatorState

from quadrotor.renderer import animate_k3d

def run_and_render(initial_state: QuadrotorState, t_total: int = 0.5):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=HoveringController(),  # Using the new controller aka 'hovering controller'
        trajectory=EmptyTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [10]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 5.0)

/Users/maryamchoudhury/.local/share/virtualenvs/oa-project-08-24-RgAkkMD-/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()